In [ ]:
import copy
# set env LD_LIBRARY_PATH to install_path
import os
install_path = "/Users/yumengxu/Project/Physics/cwb/cwb_source/tools/install/lib"
# install_path = "/Users/yumengxu/Project/Physics/cwb/cwb_O4/cwb/tools/install/lib"
os.environ['LD_LIBRARY_PATH'] = install_path

# import ROOT
#
# ROOT.gSystem.Load("wavelet")

# load user parameters
from pycwb.config import Config, CWBConfig
from pycwb import logger_init
cwb_config = CWBConfig('./config.ini')
cwb_config.export_to_envs()
logger_init()

config = Config('./user_parameters_mdc.yaml')

gps_end_time = 1264131194.580

In [ ]:
from pycwb.modules.data_quality import read_seg_list

seg = read_seg_list(config.dq_files, 'CWB_CAT1')

In [ ]:
from pycwb.modules.read_data import read_from_config

data = read_from_config(config)

In [ ]:
dc_data = [d.crop(0, d.end_time - d.start_time - 776) for d in data]

In [ ]:
from pycwb.modules.data_conditioning import data_conditioning
from pycwb.modules.plot import plot_spectrogram

tf_maps, nRMS = data_conditioning(config, data)

# plot_spectrogram(tf_maps[0], gwpy_plot=True)
#
# plot_spectrogram(tf_maps[1], gwpy_plot=True)
#
# plot_spectrogram(tf_maps[0], xmin=1264130816+324, xmax=1264130816+324+65, gwpy_plot=True)
# plot_spectrogram(tf_maps[1], xmin=1264130816+324, xmax=1264130816+324+65, gwpy_plot=True)

In [ ]:
# load pickled data events.pkl
import pickle
with open('events.pkl', 'rb') as f:
    events = pickle.load(f)
events

In [ ]:
i = 1
boxes = [[e.start[i], e.stop[i], e.low[i], e.high[i]] for e in events if len(e.start) > 0]

In [ ]:
import matplotlib.pyplot as plt
i = 1
plot = plot_spectrogram(tf_maps[i], xmin=1264130816, xmax=1264131592,
 figsize=(24,6), gwpy_plot=True)

# plot boxes on the plot
for box in boxes:
    ax = plot.gca()
    ax.add_patch(plt.Rectangle((box[0], box[2]), box[1] - box[0], box[3] - box[2], linewidth=0.5, fill=False, color='red'))

plot.savefig("event.png")


In [ ]:
self = events[-1]
# convert the following string to a nicer output, with 6 digits after the decimal point
output = f"""
nevent: \t {self.nevent}
ndim: \t\t {self.ndim}
run: \t\t {self.run}
rho: \t\t {self.rho[0]}
netCC: \t\t {self.netcc[0]}
netED: \t\t {" ".join(str(x) for x in self.neted)}
penalty: \t {self.penalty}
gnet: \t\t {self.gnet}
anet: \t\t {self.anet}
inet: \t\t {self.inet}
likelihood:  {self.likelihood}
ecor: \t\t {self.ecor}
ECOR: \t\t {self.ECOR}
factor: \t {self.factor}
range: \t\t {self.range}
mchirp: \t\t {self.chirp}
norm: \t\t {self.norm}
usize: \t\t {self.usize}

ifo: \t\t {self.ifo_list}
eventID: \t\t {self.eventID}
rho: \t\t {self.rho}
type: \t\t {self.type}
rate: \t\t {self.rate}
volume: \t\t {self.volume}
size: \t\t {self.size}
lag: \t\t {self.lag}
slag: \t\t {self.slag}
phi: \t\t {self.phi}
theta: \t\t {self.theta}
psi: \t\t {self.psi}
iota: \t\t {self.iota}
bp: \t\t {self.bp}
bx: \t\t {self.bx}
chirp: \t\t {self.chirp}
range: \t\t {self.range}
Deff: \t\t {self.Deff}
mass: \t\t {self.mass}
spin: \t\t {self.spin}
eBBH: \t\t {self.eBBH}
null: \t\t {self.null}
strain: \t\t {self.strain}
hrss: \t\t {self.hrss}
noise: \t\t {self.noise}

start: \t\t {self.start}
stop: \t\t {self.stop}
left: \t\t {self.left}
right: \t\t {self.right}
duration: \t\t {self.duration}
frequency: \t\t {self.frequency}
low: \t\t {self.low}
high: \t\t {self.high}
bandwidth: \t\t {self.bandwidth}

snr: \t\t {self.snr}
xSNR: \t\t {self.xSNR}
sSNR: \t\t {self.sSNR}
iSNR: \t\t {self.iSNR}
oSNR: \t\t {self.oSNR}
ioSNR: \t\t {self.ioSNR}

netcc: \t\t {self.netcc}
neted: \t\t {self.neted}
        """

print(output)

# save to event.txt
with open('event.txt', 'w') as f:
    f.write(output)

In [ ]:
# gps_end_time_list = [1264131046.0, 1264130902.25, 1264131141.96875, 1264131167.09375, 1264131168.03125, 1264131204.5, 1264131296.375,]
# gps_end_time_list.sort()
# print(gps_end_time_list)
# for t in gps_end_time_list:
#     plot_spectrogram(tf_maps[0], xmin=t-30, xmax=t+30, gwpy_plot=True)

In [ ]:
# initialize network
from pycwb.modules.coherence import create_network
net, wdm_list = create_network(1, config, tf_maps, nRMS)

In [ ]:
# from pycwb.constants import WDM_BETAORDER, WDM_PRECISION
#
# net.wdmMRA.BetaOrder, WDM_BETAORDER, WDM_PRECISION, net.wdmMRA.precision

# [w.BetaOrder for w in net.wdmList]
import ROOT
import numpy as np

# up_n = config.rateANA // 1024
# m_tau = net.getDelay('MAX')
# this = ROOT.WSeries(np.double)(tf_maps[0])
# ts = ROOT.wavearray(np.double)(tf_maps[0])
# w = wdm_list[0]
#
# xx = ROOT.wavearray(np.double)
# tmp = ROOT.WSeries(np.double)()
#
# xx = ts

# this.Forward(ts, w, 0)
# tmp.Forward(ts,w)
# tmp.setlow(this.getlow())
# tmp.sethigh(this.gethigh())
# shape=tmp.wdmPacket(net.pattern,'E')
# print(shape)
# this.max(tmp)
# print(ts.getMaxLevel(), ts.getlow(), ts.gethigh())
# ts.Forward(ts, w, 0)
# print(ts.getMaxLevel(), ts.getlow(), ts.gethigh())
# ts.Forward(ts, wdm_list[1], 0)
# print(ts.getMaxLevel(), ts.getlow(), ts.gethigh())
# ts.Forward(ts, wdm_list[2], 0)
# print(ts.getMaxLevel(), ts.getlow(), ts.gethigh())

## copy network
# import copy
# net2 = copy.deepcopy(net)
# print(net.pattern, net2.pattern)
# net2.pattern = 1
# print(net.pattern, net2.pattern)

In [ ]:
from pycwb.modules.coherence import coherence, coherence_parallel
sparse_table_list, cluster_list = coherence(config, net, tf_maps, wdm_list)

In [ ]:
from pycwb.modules.super_cluster import supercluster
cluster, pwc_list = supercluster(config, net, wdm_list, cluster_list, sparse_table_list)

In [ ]:
# start_net = np.array(pwc_list[0].get("start", 0, 'L', 0))
# stop_net = np.array(pwc_list[0].get("stop", 0, 'L', 0))
# list(zip(start_net, stop_net - start_net, stop_net))

In [ ]:
# list(pwc_list[0].cList[0])
# pixels = [pwc_list[0].pList[i] for i in pwc_list[0].cList[1]]
# pwc = net.getwc(0)
# pwc.clear()
# def select_clusters(cluster, c, cluster_id):
#     new_cluster_id = 1
#     clist = c.cList[cluster_id]
#     for pixel in c.pList[clist[0]: clist[-1] + 1]:
#         pixel.clusterID = new_cluster_id
#         cluster.pList.push_back(pixel)
#     cluster.sCuts.push_back(0)
#     cluster.cList.push_back(range(clist.size()))
#     cluster.cRate.push_back(c.cRate[cluster_id])
#     cluster.cTime.push_back(c.cTime[cluster_id])
#     cluster.cFreq.push_back(c.cFreq[cluster_id])
#     cluster.sArea.push_back(c.sArea[cluster_id])
#     cluster.p_Map.push_back(c.p_Map[cluster_id])
#     cluster.nTofF.push_back(c.nTofF[cluster_id])
#     cluster.p_Ind.push_back(c.p_Ind[cluster_id])
#
# select_clusters(pwc, pwc_list[0], 1)

In [ ]:
from pycwb.modules.netcluster import select_clusters
# j = 0
#
# pwc = net.getwc(j)
#
# k = 0
# pwc.cpf(pwc_list[0])
# pwc.clean()
# # select_clusters(pwc, pwc_list[j], k)
# # pwc.print()
# # pwc.setcore(False, 1)
# # pwc.loadTDampSSE(net, 'a', config.BATCH, config.BATCH)
# # lag = 0
# # ID = 0
# # net.MRA = True
# # net.likelihoodWP(config.search, lag, ID, ROOT.nullptr, config.Search)
# pwc.cData.size()

In [ ]:
from pycwb.modules.likelihood import likelihood
# import cppyy.ll
# cppyy.ll.set_signals_as_exception(True)
# with cppyy.ll.signals_as_exception():
events = likelihood(config, net, sparse_table_list, pwc_list, cluster, wdm_list)

In [ ]:
# pwc = net.getwc(0)
# start_net = np.array(pwc.get("start", 0, 'L', 0))
# stop_net = np.array(pwc.get("stop", 0, 'L', 0))
# list(zip(start_net, stop_net - start_net, stop_net))

In [ ]:
for event in events:
    print(event.start, event.stop, event.duration)
    print(event.rho, event.likelihood)

In [ ]:
pwc = net.getwc(0)
for pcd in pwc.cData:
    print(pcd.netRHO)

In [ ]:
#pwc.get((char*)"ID",0,'S',0)
list(pwc.get("ID", 0, 'S', 0))

In [ ]:

# from pycwb.modules.netpixel import get_time_frequency
# points = [get_time_frequency(cluster_list[0],p) for c in cluster_list for p in c.pList]
#
# import matplotlib.pyplot as plt
# plt.figure(figsize=(12,6))
# plt.scatter([p[0] for p in points], [p[1] for p in points])
# # xmin=gps_end_time-20, xmax=gps_end_time+30
# plt.xlim((gps_end_time-20, gps_end_time+30))
# plt.ylim((0,1000))